<h2 id="exercises">Exercises</h2>
<p>The end result of this exercise should be a Jupyter notebook named <code>model</code>.</p>
<p>Using <a href="https://ds.codeup.com/saas.csv">saas.csv</a> or log data from API usage or store_item_sales</p>
<ol>


1. Split data (train/validate/test) and resample by any period, except daily, and aggregate using the sum. </li>


2. Forecast, plot and evaluate using each of the 4 parametric based methods we discussed:<ul>
<li>Simple Average</li>


<li>Moving Average</li>


<li>Holt's Linear Trend Model</li>


<li>Based on previous year/month/etc., this is up to you.</li>


<p>Optional: Using store item demand</p>


<li>Predict 2018 total <strong>monthly</strong> sales for a single store and/or item by creating a model.</li>


<li>Return a dataframe with the month, store_id, y-hat, and the confidence intervals (y-hat lower, y-hat upper).</li>


<li>Plot the 2018 monthly sales predictions.</li>
</ol>